## SETUP 

In [1]:
#import libraries
import pandas as pd
import sqlite3

In [4]:
#df = pd.DataFrame(#data here :/)

df = pd.DataFrame({'name': ['Juan', 'Victoria', 'Mary'], \
'age': [23, 34, 43], 'city': ['Miami', 'Buenos Aries', 'Santiago']})


In [5]:
df

name  age          city
0      Juan   23         Miami
1  Victoria   34  Buenos Aries
2      Mary   43      Santiago

In [7]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_tutorial.db')

In [11]:
df.to_sql('people', cnn)

In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///jupyter_sql_tutorial.db

In [17]:
%%sql
SELECT *
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


[(0, 'Juan', 23, 'Miami'),
 (1, 'Victoria', 34, 'Buenos Aries'),
 (2, 'Mary', 43, 'Santiago')]

In [18]:
%%sql
SELECT count(*)
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


[(3,)]

In [19]:
%%sql
SELECT sum(age) as 'age_sum'
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


[(100,)]

## Parameters walkthrough

In [20]:
#create dummy dataframe
df = pd.DataFrame({'transaction_id': ['9', '8', '7', '6', '5', '4', '3'], \
                  'user_id': ['rafa', 'roy', 'kenny', 'brendan', 'jurgen', 'roy', 'roy'],\
                  'transaction_date': ['2021-12-21', '2020-12-21', '2019-12-21',\
                                       '2018-11-21', '2017-10-21', '2019-03-02', '2010-01-01'],\
                  'amount': ['10', '15', '20', '24', '25', '31', '42']})

In [21]:
df

transaction_id  user_id transaction_date amount
0              9     rafa       2021-12-21     10
1              8      roy       2020-12-21     15
2              7    kenny       2019-12-21     20
3              6  brendan       2018-11-21     24
4              5   jurgen       2017-10-21     25
5              4      roy       2019-03-02     31
6              3      roy       2010-01-01     42

In [22]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('dummy.db')

In [29]:
df.to_sql('managers1', cnn)

In [30]:
%reload_ext sql

In [31]:
%sql sqlite:///dummy.db

In [32]:
%%sql
SELECT *
FROM managers1

 * sqlite:///dummy.db
Done.


[(0, '9', 'rafa', '2021-12-21', '10'),
 (1, '8', 'roy', '2020-12-21', '15'),
 (2, '7', 'kenny', '2019-12-21', '20'),
 (3, '6', 'brendan', '2018-11-21', '24'),
 (4, '5', 'jurgen', '2017-10-21', '25'),
 (5, '4', 'roy', '2019-03-02', '31'),
 (6, '3', 'roy', '2010-01-01', '42')]

In [34]:
%%sql
SELECT sum(amount) as 'spend_sum'
FROM managers1

 * sqlite:///dummy.db
Done.


[(167,)]

In [35]:
%%sql
SELECT 
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
FROM
    managers1
WHERE
    user_id = 'roy'
    and transaction_date = '2019-03-02'

 * sqlite:///dummy.db
Done.


[('roy', 1, 31)]

## Pandas solution

ovechal01, ovi8

1228 * 11,


In [2]:
#import libraries
from datetime import timedelta
from dateutil.parser import parse
import pandas as pd
import sqlite3
import time

In [3]:
stat_handle = input('stat handle:')
stat_sheet = f'./data/{stat_handle}_stats.csv'
stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

stat handle:ovechal01


In [4]:
stat_df

date  goals  assists  +/-  pim  shots  shot%    TOI  \
0    2013-01-19 19:00:00-05:00      0        0    0    2      4    0.0  23:35   
1    2013-01-22 19:00:00-05:00      0        1    0    0      4    0.0  23:59   
2    2013-01-24 19:00:00-05:00      0        0   -1    2      3    0.0  22:26   
3    2013-01-25 19:00:00-05:00      0        0    0    0      2    0.0  22:29   
4    2013-01-27 15:00:00-05:00      1        0   -1    0      5   20.0  18:29   
..                         ...    ...      ...  ...  ...    ...    ...    ...   
675  2021-12-29 19:00:00-05:00      0        1    1    0      4    0.0  26:01   
676  2021-12-31 19:30:00-05:00      2        0    2    0      7   28.6  19:54   
677  2022-01-02 15:00:00-05:00      0        0   -2    0      3    0.0  22:56   
678  2022-01-07 20:00:00-05:00      0        0   -3    0      3    0.0  20:55   
679  2022-01-08 20:00:00-05:00      0        1   -1    0      4    0.0  23:17   

     hits  blocks  faceoff %  
0       2       0        NaN  
1       4       0        NaN  
2       1       0        NaN  
3       1       0        NaN  
4       3       0        NaN  
..    ...     ...        ...  
675     0       0        NaN  
676     1       0        NaN  
677     4       0      100.0  
678     1       0        NaN  
679     0       0        NaN  

[680 rows x 11 columns]

In [6]:
def find_dates(player):
    dates = []
    stat_handle = player
    stat_sheet = f'./data/{stat_handle}_stats.csv'
    stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)
    for row in stat_df.iterrows():
        values = row[1]
        pd_date = parse(values[0])
        
        #time delta to 1 day prior, 
        #worth investigating the effect of using time delta to travel to midnight on gameday
        start_date = pd_date - timedelta(days = 1)
        time_range = (start_date, pd_date)
        #append to dates list
        dates.append(time_range)
    return dates
        
dates = find_dates('ovechal01')
for date in dates:
    print(date[0], '**', date[1])

2013-01-18 19:00:00-05:00 ** 2013-01-19 19:00:00-05:00
2013-01-21 19:00:00-05:00 ** 2013-01-22 19:00:00-05:00
2013-01-23 19:00:00-05:00 ** 2013-01-24 19:00:00-05:00
2013-01-24 19:00:00-05:00 ** 2013-01-25 19:00:00-05:00
2013-01-26 15:00:00-05:00 ** 2013-01-27 15:00:00-05:00
2013-01-28 19:30:00-05:00 ** 2013-01-29 19:30:00-05:00
2013-01-30 19:00:00-05:00 ** 2013-01-31 19:00:00-05:00
2013-01-31 19:00:00-05:00 ** 2013-02-01 19:00:00-05:00
2013-02-03 00:30:00-05:00 ** 2013-02-04 00:30:00-05:00
2013-02-04 19:00:00-05:00 ** 2013-02-05 19:00:00-05:00
2013-02-06 19:00:00-05:00 ** 2013-02-07 19:00:00-05:00
2013-02-08 19:00:00-05:00 ** 2013-02-09 19:00:00-05:00
2013-02-11 19:30:00-05:00 ** 2013-02-12 19:30:00-05:00
2013-02-13 19:30:00-05:00 ** 2013-02-14 19:30:00-05:00
2013-02-16 18:00:00-05:00 ** 2013-02-17 18:00:00-05:00
2013-02-20 19:00:00-05:00 ** 2013-02-21 19:00:00-05:00
2013-02-23 00:00:00-05:00 ** 2013-02-24 00:00:00-05:00
2013-02-25 19:00:00-05:00 ** 2013-02-26 19:00:00-05:00
2013-02-26

2016-10-25 21:30:00-04:00 ** 2016-10-26 21:30:00-04:00
2016-10-28 22:00:00-04:00 ** 2016-10-29 22:00:00-04:00
2016-10-29 21:30:00-04:00 ** 2016-10-30 21:30:00-04:00
2016-10-31 20:00:00-04:00 ** 2016-11-01 20:00:00-04:00
2016-11-02 19:00:00-04:00 ** 2016-11-03 19:00:00-04:00
2016-11-04 19:00:00-04:00 ** 2016-11-05 19:00:00-04:00
2016-11-07 19:00:00-05:00 ** 2016-11-08 19:00:00-05:00
2016-11-10 20:30:00-05:00 ** 2016-11-11 20:30:00-05:00
2016-11-11 19:00:00-05:00 ** 2016-11-12 19:00:00-05:00
2016-11-14 19:00:00-05:00 ** 2016-11-15 19:00:00-05:00
2016-11-15 19:30:00-05:00 ** 2016-11-16 19:30:00-05:00
2016-11-17 19:00:00-05:00 ** 2016-11-18 19:00:00-05:00
2016-11-20 00:30:00-05:00 ** 2016-11-21 00:30:00-05:00
2016-11-22 19:00:00-05:00 ** 2016-11-23 19:00:00-05:00
2016-11-24 17:00:00-05:00 ** 2016-11-25 17:00:00-05:00
2016-11-25 19:00:00-05:00 ** 2016-11-26 19:00:00-05:00
2016-11-30 19:00:00-05:00 ** 2016-12-01 19:00:00-05:00
2016-12-02 19:00:00-05:00 ** 2016-12-03 19:00:00-05:00
2016-12-04

2021-03-19 19:00:00-04:00 ** 2021-03-20 19:00:00-04:00
2021-03-24 19:00:00-04:00 ** 2021-03-25 19:00:00-04:00
2021-03-25 19:00:00-04:00 ** 2021-03-26 19:00:00-04:00
2021-03-28 00:00:00-04:00 ** 2021-03-29 00:00:00-04:00
2021-03-29 19:00:00-04:00 ** 2021-03-30 19:00:00-04:00
2021-03-31 19:00:00-04:00 ** 2021-04-01 19:00:00-04:00
2021-04-01 19:00:00-04:00 ** 2021-04-02 19:00:00-04:00
2021-04-03 15:00:00-04:00 ** 2021-04-04 15:00:00-04:00
2021-04-05 19:00:00-04:00 ** 2021-04-06 19:00:00-04:00
2021-04-07 19:00:00-04:00 ** 2021-04-08 19:00:00-04:00
2021-04-08 19:00:00-04:00 ** 2021-04-09 19:00:00-04:00
2021-04-10 19:00:00-04:00 ** 2021-04-11 19:00:00-04:00
2021-04-12 19:00:00-04:00 ** 2021-04-13 19:00:00-04:00
2021-04-14 19:00:00-04:00 ** 2021-04-15 19:00:00-04:00
2021-04-17 00:30:00-04:00 ** 2021-04-18 00:30:00-04:00
2021-04-18 00:00:00-04:00 ** 2021-04-19 00:00:00-04:00
2021-04-21 19:00:00-04:00 ** 2021-04-22 19:00:00-04:00
2021-05-02 19:00:00-04:00 ** 2021-05-03 19:00:00-04:00
2021-05-10

In [7]:
stat_handle = input('twitter handle:')
stat_sheet = f'./data/{stat_handle}.csv'
twitter_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

twitter handle:ovi8


In [8]:
twitter_df

date time   display name  \
0    2021-12-02 18:19:51-05:00  Alex Ovechkin   
1    2021-11-20 20:44:54-05:00  Alex Ovechkin   
2    2021-11-16 04:04:50-05:00  Alex Ovechkin   
3    2021-11-04 13:16:26-04:00  Alex Ovechkin   
4    2021-10-28 22:15:31-04:00  Alex Ovechkin   
..                         ...            ...   
620  2009-01-21 14:51:20-05:00  Alex Ovechkin   
621  2009-01-20 03:40:04-05:00  Alex Ovechkin   
622  2009-01-20 01:35:55-05:00  Alex Ovechkin   
623  2009-01-19 04:31:30-05:00  Alex Ovechkin   
624  2009-01-17 20:46:40-05:00  Alex Ovechkin   

                                               content  
0    The 2022 #CapsCanineCalendar is Ovi and Blake ...  
1    🏆 You took the biggest step!!! Congrats @WashS...  
2    Great to be part of the first @blockassetco Le...  
3    Good luck and see you guys soon in DC!! https:...  
4    Very proud to be a @Blockassetco Legend with @...  
..                                                 ...  
620  At DC airport with tons of obama fans. Taking ...  
621  Lots of questions about my trick shot plans - ...  
622  Only thing better than scoring goals is winnin...  
623  Big win last night on a sick shot by Semin. No...  
624  Thought I'd use this to give you quick updates...  

[625 rows x 3 columns]

In [10]:
with open('ovechkin_dates.csv', 'w', encoding='utf-8'):
    point_list = []
    twitter_handle = 'ovi8'
    tweet_sheet = f'./data/{twitter_handle}.csv'
    twitter_df = pd.read_csv(tweet_sheet, sep= '\t', header= 0, index_col= None)

    dates = find_dates('ovechal01')
    for date in dates:
        start = date[0]
        end = date[1]
        #Comprehension likely goes here, or at least will affect the line below
        for row in twitter_df['date time']:
            row = parse(row)
            if row >= start and row <= end:
                #add point to new DB
                point_list.append(row) 
                #print(f'point found: {row}')
                
print(f'{len(point_list)} points found')

73 points found


In [13]:
tweet_dates = twitter_df.iloc[:,0]
for d in tweet_dates:
    print(d)

2021-12-02 18:19:51-05:00
2021-11-20 20:44:54-05:00
2021-11-16 04:04:50-05:00
2021-11-04 13:16:26-04:00
2021-10-28 22:15:31-04:00
2021-10-12 22:41:50-04:00
2021-08-07 19:00:04-04:00
2021-07-30 17:38:23-04:00
2021-07-27 17:01:06-04:00
2021-03-17 22:47:01-04:00
2021-03-04 23:43:43-05:00
2020-12-17 21:27:52-05:00
2020-12-03 18:31:04-05:00
2020-10-16 16:33:52-04:00
2020-10-08 17:29:38-04:00
2020-10-02 16:12:24-04:00
2020-09-03 14:11:57-04:00
2020-08-24 17:54:06-04:00
2020-06-10 00:11:30-04:00
2020-06-02 01:58:22-04:00
2020-04-30 17:32:52-04:00
2020-04-23 02:32:56-04:00
2020-04-22 01:53:32-04:00
2020-04-19 23:35:52-04:00
2020-04-07 21:21:04-04:00
2020-01-26 23:04:19-05:00
2020-01-14 04:27:02-05:00
2020-01-11 00:15:14-05:00
2019-12-22 18:18:40-05:00
2019-12-13 16:45:43-05:00
2019-11-29 15:38:54-05:00
2019-11-22 21:29:21-05:00
2019-11-21 21:18:27-05:00
2019-11-20 02:20:09-05:00
2019-10-30 20:34:39-04:00
2019-10-11 21:21:43-04:00
2019-10-10 05:02:29-04:00
2019-10-08 02:16:31-04:00
2019-10-01 1

In [11]:
for p in point_list:
    print(p)

2013-03-07 17:22:54-05:00
2013-03-07 17:20:57-05:00
2013-03-14 02:57:16-04:00
2013-03-14 00:04:50-04:00
2013-03-14 00:00:46-04:00
2013-03-22 18:07:31-04:00
2013-03-22 18:04:13-04:00
2013-03-22 18:03:07-04:00
2013-03-24 12:54:47-04:00
2013-03-24 01:34:46-04:00
2013-03-24 01:30:36-04:00
2013-03-24 00:03:45-04:00
2013-03-23 19:03:04-04:00
2013-03-31 03:18:19-04:00
2013-04-04 15:48:15-04:00
2013-04-03 21:14:43-04:00
2013-04-05 23:35:23-04:00
2013-04-05 23:34:04-04:00
2013-04-07 15:00:57-04:00
2013-04-10 21:16:38-04:00
2013-04-10 20:39:59-04:00
2013-04-10 20:17:58-04:00
2013-04-17 19:46:44-04:00
2013-04-17 19:45:26-04:00
2013-04-17 19:44:16-04:00
2013-04-17 19:44:10-04:00
2013-04-24 19:12:31-04:00
2013-10-02 21:03:11-04:00
2013-10-02 20:58:44-04:00
2013-10-05 15:02:19-04:00
2013-11-09 02:17:13-05:00
2013-11-12 00:01:28-05:00
2013-11-27 00:55:33-05:00
2013-11-29 02:29:07-05:00
2013-12-23 00:37:46-05:00
2014-01-11 19:13:58-05:00
2014-02-08 03:19:33-05:00
2014-03-05 21:36:10-05:00
2014-03-15 2

In [ ]:
with open('ovechkin_dates.csv', 'w', encoding='utf-8'):
    point_list = []
    twitter_handle = 'ovi8'
    tweet_sheet = f'./data/{twitter_handle}.csv'
    twitter_df = pd.read_csv(tweet_sheet, sep= '\t', header= 0, index_col= None)
    time = pd.date_range(start = (x), end= (y), freq = 'M')

    dates = find_dates('ovechal01')
    for date in dates:
        start = date[0]
        print('end', type(start))
        #start = parse(start)
        end = date[1]
        #end = parse(end)
        print('end', type(end))
        
        #Comprehension likely goes here, or at least will affect the line below
        
        for row in twitter_df['date time']:
            for r in twitter_df:
                print(row, type(row))
                try:
                    row = parse(row)
                    print(r)
                
                    if row >= start and row <= end:
                        #add point to new DB
                        if row in point_list:
                            break
                        else:
                            point_list.append(r) 
                            #print(f'point found: {row}')
                except:
                    continue
        
                
print(f'{len(point_list)} points found')

end <class 'datetime.datetime'>
end <class 'datetime.datetime'>
2021-12-02 18:19:51-05:00 <class 'str'>
date time
2021-12-02 18:19:51-05:00 <class 'datetime.datetime'>
2021-12-02 18:19:51-05:00 <class 'datetime.datetime'>
2021-11-20 20:44:54-05:00 <class 'str'>
date time
2021-11-20 20:44:54-05:00 <class 'datetime.datetime'>
2021-11-20 20:44:54-05:00 <class 'datetime.datetime'>
2021-11-16 04:04:50-05:00 <class 'str'>
date time
2021-11-16 04:04:50-05:00 <class 'datetime.datetime'>
2021-11-16 04:04:50-05:00 <class 'datetime.datetime'>
2021-11-04 13:16:26-04:00 <class 'str'>
date time
2021-11-04 13:16:26-04:00 <class 'datetime.datetime'>
2021-11-04 13:16:26-04:00 <class 'datetime.datetime'>
2021-10-28 22:15:31-04:00 <class 'str'>
date time
2021-10-28 22:15:31-04:00 <class 'datetime.datetime'>
2021-10-28 22:15:31-04:00 <class 'datetime.datetime'>
2021-10-12 22:41:50-04:00 <class 'str'>
date time
2021-10-12 22:41:50-04:00 <class 'datetime.datetime'>
2021-10-12 22:41:50-04:00 <class 'datetime

date time
2015-09-11 21:32:23-04:00 <class 'datetime.datetime'>
2015-09-11 21:32:23-04:00 <class 'datetime.datetime'>
2015-04-29 16:19:58-04:00 <class 'str'>
date time
2015-04-29 16:19:58-04:00 <class 'datetime.datetime'>
2015-04-29 16:19:58-04:00 <class 'datetime.datetime'>
2015-04-03 20:38:08-04:00 <class 'str'>
date time
2015-04-03 20:38:08-04:00 <class 'datetime.datetime'>
2015-04-03 20:38:08-04:00 <class 'datetime.datetime'>
2015-03-27 20:33:01-04:00 <class 'str'>
date time
2015-03-27 20:33:01-04:00 <class 'datetime.datetime'>
2015-03-27 20:33:01-04:00 <class 'datetime.datetime'>
2015-03-02 04:56:35-05:00 <class 'str'>
date time
2015-03-02 04:56:35-05:00 <class 'datetime.datetime'>
2015-03-02 04:56:35-05:00 <class 'datetime.datetime'>
2015-01-26 02:31:38-05:00 <class 'str'>
date time
2015-01-26 02:31:38-05:00 <class 'datetime.datetime'>
2015-01-26 02:31:38-05:00 <class 'datetime.datetime'>
2015-01-25 18:55:33-05:00 <class 'str'>
date time
2015-01-25 18:55:33-05:00 <class 'datetime

2014-06-22 09:32:44-04:00 <class 'str'>
date time
2014-06-22 09:32:44-04:00 <class 'datetime.datetime'>
2014-06-22 09:32:44-04:00 <class 'datetime.datetime'>
2014-06-22 09:19:13-04:00 <class 'str'>
date time
2014-06-22 09:19:13-04:00 <class 'datetime.datetime'>
2014-06-22 09:19:13-04:00 <class 'datetime.datetime'>
2014-06-22 05:21:17-04:00 <class 'str'>
date time
2014-06-22 05:21:17-04:00 <class 'datetime.datetime'>
2014-06-22 05:21:17-04:00 <class 'datetime.datetime'>
2014-06-21 19:37:38-04:00 <class 'str'>
date time
2014-06-21 19:37:38-04:00 <class 'datetime.datetime'>
2014-06-21 19:37:38-04:00 <class 'datetime.datetime'>
2014-06-21 10:02:50-04:00 <class 'str'>
date time
2014-06-21 10:02:50-04:00 <class 'datetime.datetime'>
2014-06-21 10:02:50-04:00 <class 'datetime.datetime'>
2014-06-21 07:52:31-04:00 <class 'str'>
date time
2014-06-21 07:52:31-04:00 <class 'datetime.datetime'>
2014-06-21 07:52:31-04:00 <class 'datetime.datetime'>
2014-06-20 12:18:17-04:00 <class 'str'>
date time
20

2013-09-29 16:13:16-04:00 <class 'datetime.datetime'>
2013-09-29 16:12:46-04:00 <class 'str'>
date time
2013-09-29 16:12:46-04:00 <class 'datetime.datetime'>
2013-09-29 16:12:46-04:00 <class 'datetime.datetime'>
2013-09-29 16:11:58-04:00 <class 'str'>
date time
2013-09-29 16:11:58-04:00 <class 'datetime.datetime'>
2013-09-29 16:11:58-04:00 <class 'datetime.datetime'>
2013-09-29 16:08:53-04:00 <class 'str'>
date time
2013-09-29 16:08:53-04:00 <class 'datetime.datetime'>
2013-09-29 16:08:53-04:00 <class 'datetime.datetime'>
2013-09-29 16:07:46-04:00 <class 'str'>
date time
2013-09-29 16:07:46-04:00 <class 'datetime.datetime'>
2013-09-29 16:07:46-04:00 <class 'datetime.datetime'>
2013-09-29 10:56:36-04:00 <class 'str'>
date time
2013-09-29 10:56:36-04:00 <class 'datetime.datetime'>
2013-09-29 10:56:36-04:00 <class 'datetime.datetime'>
2013-09-29 09:35:01-04:00 <class 'str'>
date time
2013-09-29 09:35:01-04:00 <class 'datetime.datetime'>
2013-09-29 09:35:01-04:00 <class 'datetime.datetime'

2013-04-07 15:00:57-04:00 <class 'datetime.datetime'>
2013-04-07 15:00:57-04:00 <class 'datetime.datetime'>
2013-04-05 23:35:23-04:00 <class 'str'>
date time
2013-04-05 23:35:23-04:00 <class 'datetime.datetime'>
2013-04-05 23:35:23-04:00 <class 'datetime.datetime'>
2013-04-05 23:34:04-04:00 <class 'str'>
date time
2013-04-05 23:34:04-04:00 <class 'datetime.datetime'>
2013-04-05 23:34:04-04:00 <class 'datetime.datetime'>
2013-04-04 15:48:15-04:00 <class 'str'>
date time
2013-04-04 15:48:15-04:00 <class 'datetime.datetime'>
2013-04-04 15:48:15-04:00 <class 'datetime.datetime'>
2013-04-03 21:14:43-04:00 <class 'str'>
date time
2013-04-03 21:14:43-04:00 <class 'datetime.datetime'>
2013-04-03 21:14:43-04:00 <class 'datetime.datetime'>
2013-04-01 18:12:34-04:00 <class 'str'>
date time
2013-04-01 18:12:34-04:00 <class 'datetime.datetime'>
2013-04-01 18:12:34-04:00 <class 'datetime.datetime'>
2013-03-31 03:18:19-04:00 <class 'str'>
date time
2013-03-31 03:18:19-04:00 <class 'datetime.datetime'

2011-09-18 01:26:02-04:00 <class 'str'>
date time
2011-09-18 01:26:02-04:00 <class 'datetime.datetime'>
2011-09-18 01:26:02-04:00 <class 'datetime.datetime'>
2011-09-17 17:10:53-04:00 <class 'str'>
date time
2011-09-17 17:10:53-04:00 <class 'datetime.datetime'>
2011-09-17 17:10:53-04:00 <class 'datetime.datetime'>
2011-09-12 13:06:47-04:00 <class 'str'>
date time
2011-09-12 13:06:47-04:00 <class 'datetime.datetime'>
2011-09-12 13:06:47-04:00 <class 'datetime.datetime'>
2011-09-09 20:10:38-04:00 <class 'str'>
date time
2011-09-09 20:10:38-04:00 <class 'datetime.datetime'>
2011-09-09 20:10:38-04:00 <class 'datetime.datetime'>
2011-09-09 18:24:14-04:00 <class 'str'>
date time
2011-09-09 18:24:14-04:00 <class 'datetime.datetime'>
2011-09-09 18:24:14-04:00 <class 'datetime.datetime'>
2011-09-09 03:47:07-04:00 <class 'str'>
date time
2011-09-09 03:47:07-04:00 <class 'datetime.datetime'>
2011-09-09 03:47:07-04:00 <class 'datetime.datetime'>
2011-09-09 02:33:05-04:00 <class 'str'>
date time
20

2009-01-30 20:11:16-05:00 <class 'datetime.datetime'>
2009-01-30 20:11:16-05:00 <class 'datetime.datetime'>
2009-01-30 20:07:38-05:00 <class 'str'>
date time
2009-01-30 20:07:38-05:00 <class 'datetime.datetime'>
2009-01-30 20:07:38-05:00 <class 'datetime.datetime'>
2009-01-30 20:06:56-05:00 <class 'str'>
date time
2009-01-30 20:06:56-05:00 <class 'datetime.datetime'>
2009-01-30 20:06:56-05:00 <class 'datetime.datetime'>
2009-01-30 20:05:54-05:00 <class 'str'>
date time
2009-01-30 20:05:54-05:00 <class 'datetime.datetime'>
2009-01-30 20:05:54-05:00 <class 'datetime.datetime'>
2009-01-30 20:03:52-05:00 <class 'str'>
date time
2009-01-30 20:03:52-05:00 <class 'datetime.datetime'>
2009-01-30 20:03:52-05:00 <class 'datetime.datetime'>
2009-01-30 20:02:59-05:00 <class 'str'>
date time
2009-01-30 20:02:59-05:00 <class 'datetime.datetime'>
2009-01-30 20:02:59-05:00 <class 'datetime.datetime'>
2009-01-30 20:02:39-05:00 <class 'str'>
date time
2009-01-30 20:02:39-05:00 <class 'datetime.datetime'

In [12]:
with open('ovechkin_dates.csv', 'w', encoding='utf-8'):
    point_list = []
    twitter_handle = 'ovi8'
    tweet_sheet = f'./data/{twitter_handle}.csv'
    twitter_df = pd.read_csv(tweet_sheet, sep= '\t', header= 0, index_col= None)

    dates = find_dates('ovechal01')
    #Trying to implement df.iterrows()
    for ind in twitter_df.index:
        print(ind)
    
    for date in dates:
        start = date[0]
        #print('end', type(start))
        #start = parse(start)
        end = date[1]
        #end = parse(end)
        #print('end', type(end))
        
        
        '''
        for row in twitter_df['date time']:
            for r in twitter_df:
                print(row, type(row))
                try:
                    row = parse(row)
                    print(r)
                
                    if row >= start and row <= end:
                        #add point to new DB
                        if row in point_list:
                            break
                        else:
                            point_list.append(r) 
                            #print(f'point found: {row}')
                except:
                    continue
        '''
                
print(f'{len(point_list)} points found')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
18

421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64

180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429


530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
18

...

374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623


11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
2

509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
16

621
622
623
624
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
27

KeyboardInterrupt: 

Will need to set up a comprehension here, to check tweet dates for datetimes already identified as valid points of data. As the list only currently contains the datetimes, without text

In [12]:
for point in point_list:
    print(point)

2013-03-07 17:22:54-05:00
2013-03-07 17:20:57-05:00
2013-03-14 02:57:16-04:00
2013-03-14 00:04:50-04:00
2013-03-14 00:00:46-04:00
2013-03-22 18:07:31-04:00
2013-03-22 18:04:13-04:00
2013-03-22 18:03:07-04:00
2013-03-24 12:54:47-04:00
2013-03-24 01:34:46-04:00
2013-03-24 01:30:36-04:00
2013-03-24 00:03:45-04:00
2013-03-23 19:03:04-04:00
2013-03-31 03:18:19-04:00
2013-04-04 15:48:15-04:00
2013-04-03 21:14:43-04:00
2013-04-05 23:35:23-04:00
2013-04-05 23:34:04-04:00
2013-04-07 15:00:57-04:00
2013-04-10 21:16:38-04:00
2013-04-10 20:39:59-04:00
2013-04-10 20:17:58-04:00
2013-04-17 19:46:44-04:00
2013-04-17 19:45:26-04:00
2013-04-17 19:44:16-04:00
2013-04-17 19:44:10-04:00
2013-04-24 19:12:31-04:00
2013-10-02 21:03:11-04:00
2013-10-02 20:58:44-04:00
2013-10-05 15:02:19-04:00
2013-11-09 02:17:13-05:00
2013-11-12 00:01:28-05:00
2013-11-27 00:55:33-05:00
2013-11-29 02:29:07-05:00
2013-12-23 00:37:46-05:00
2014-01-11 19:13:58-05:00
2014-02-08 03:19:33-05:00
2014-03-05 21:36:10-05:00
2014-03-15 2

In [ ]:
def creating_dates(row, date_list):
    dates = list(game_windows)
    print(dates)


In [33]:
game_windows

start_date                   end_date
0    2013-01-19 19:00:00-05:00  2013-01-18 19:00:00-05:00
1    2013-01-22 19:00:00-05:00  2013-01-21 19:00:00-05:00
2    2013-01-24 19:00:00-05:00  2013-01-23 19:00:00-05:00
3    2013-01-25 19:00:00-05:00  2013-01-24 19:00:00-05:00
4    2013-01-27 15:00:00-05:00  2013-01-26 15:00:00-05:00
..                         ...                        ...
675  2021-12-29 19:00:00-05:00  2021-12-28 19:00:00-05:00
676  2021-12-31 19:30:00-05:00  2021-12-30 19:30:00-05:00
677  2022-01-02 15:00:00-05:00  2022-01-01 15:00:00-05:00
678  2022-01-07 20:00:00-05:00  2022-01-06 20:00:00-05:00
679  2022-01-08 20:00:00-05:00  2022-01-07 20:00:00-05:00

[680 rows x 2 columns]

Not catching data currently, date ranges need to be defined as demonstrated above

In [19]:
from datetime import timedelta

data_points = []
post_game = 0
tweet_dates = twitter_df.iloc[:,0]
#tweet_times = twitter_df.iloc[:,1]
for dt in tweet_dates:
    dt = pd.to_datetime(dt)
    #Use timedates to create if statements
    #Try within 24 hours of puckdrop, also from midnight on gameday
    if dt in game_list:
        gm_time = pd.to_datetime(gm_time)
        gm = pd.to_datetime(gm)
        if gm < gm_time:
            data_points.append(gm)
        else:
            post_game += 1


print(f'{len(data_points)} good points of data, {post_game} postgame tweets')

0 good points of data, 0 postgame tweets


In [9]:
for d in data_points:
    print(d)

Maybe need to include puck drop times? Would require altering stat scraper, and likely slowing it down signifigantly, to the magnitute of rendering an entire new webpage in beautfiul soup per row in the dataset. For Ovechkin this would include 1228 pages! Maybe I should use selenium as an alternative. However at least for Ovechkin, no gameday tweets were scheduled for 7pm EST. BRB Looking at tweet timestamps

In [9]:
for point in data_points:
    print(point)